In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\home laptop\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
from numpy.random import seed
seed(1)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr']]
selected_features = selected_features[selected_features['koi_disposition']!='CANDIDATE']
# selected_features['koi_disposition'] = selected_features['koi_disposition'].map({'CONFIRMED': 1, 'FALSE POSITIVE':2})
selected_features

,koi_disposition,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr
0,CONFIRMED,0.586,4.50700,874.8,2.83,443,9.11,25.8
1,FALSE POSITIVE,0.969,1.78220,10829.0,14.60,638,39.30,76.3
2,FALSE POSITIVE,1.276,2.40641,8079.2,33.46,1395,891.96,505.6
3,CONFIRMED,0.701,1.65450,603.3,2.75,1406,926.16,40.9
4,CONFIRMED,0.762,3.14020,686.0,2.77,1160,427.65,40.2
...,...,...,...,...,...,...,...,...
6983,FALSE POSITIVE,2.441,77.98300,1583.8,534.47,1006,242.54,171.5
6986,FALSE POSITIVE,0.765,4.80600,87.7,1.11,929,176.40,8.4
6987,FALSE POSITIVE,1.252,3.22210,1579.2,29.35,2088,4500.53,453.3
6989,FALSE POSITIVE,0.147,0.86500,103.6,1.07,2218,5713.41,12.3


In [6]:
# Read the training data into a dataframe
X = selected_features[['koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr']]
y = selected_features['koi_disposition']

In [7]:
#columns
feature_names = selected_features.columns[1:8]
feature_names

Index(['koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq',
       'koi_insol', 'koi_model_snr'],
      dtype='object')

# Create a Train Test Split

In [8]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split,cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, test_size=0.3)

In [9]:
X_train.head()

,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr
2681,0.7710,6.55300,62.1,0.95,691,54.06,18.8
1878,0.2810,1.74070,178.4,1.04,1731,2129.33,43.0
460,0.8660,3.08220,421.6,2.41,777,86.28,24.6
4995,0.0655,14.56000,44.7,1.50,809,101.17,5.3
5495,0.3880,3.20209,400010.0,47.81,1313,701.93,1095.9


# Pre-processing
Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
from sklearn.preprocessing import MinMaxScaler,normalize
from tensorflow.keras.utils import to_categorical

In [11]:
# Scale your data
# Create a MinMaxScaler model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# from tensorflow.keras.utils import to_categorical
# # One-hot encoding
# y_train_categorical = to_categorical(y_train)
# y_test_categorical = to_categorical(y_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, min_impurity_decrease=0.0001)
rf = rf.fit(X_train_scaled, y_train)

# Train the Model

In [14]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 0.9881465517241379
Testing Data Score: 0.9013819095477387


In [15]:
predictions = rf.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
5701,FALSE POSITIVE,FALSE POSITIVE
1407,CONFIRMED,CONFIRMED
3195,FALSE POSITIVE,FALSE POSITIVE
2848,FALSE POSITIVE,FALSE POSITIVE
2158,CONFIRMED,CONFIRMED
...,...,...
6566,FALSE POSITIVE,FALSE POSITIVE
4048,FALSE POSITIVE,FALSE POSITIVE
3430,FALSE POSITIVE,CONFIRMED
5383,FALSE POSITIVE,FALSE POSITIVE


# Hyperparameter Tuning¶
Use GridSearchCV to tune the model's parameters

In [16]:
rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0001,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [17]:
# Train the model with GridSearch
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'min_samples_split': [2, 4, 8, 16],
              'min_impurity_decrease': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [18]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

C:\Users\Home Laptop\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] min_impurity_decrease=0.0001, min_samples_split=2 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=2, score=0.893, total=   0.7s
[CV] min_impurity_decrease=0.0001, min_samples_split=2 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  min_impurity_decrease=0.0001, min_samples_split=2, score=0.921, total=   0.8s
[CV] min_impurity_decrease=0.0001, min_samples_split=2 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s


[CV]  min_impurity_decrease=0.0001, min_samples_split=2, score=0.903, total=   0.7s
[CV] min_impurity_decrease=0.0001, min_samples_split=4 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=4, score=0.894, total=   0.7s
[CV] min_impurity_decrease=0.0001, min_samples_split=4 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=4, score=0.919, total=   0.8s
[CV] min_impurity_decrease=0.0001, min_samples_split=4 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=4, score=0.901, total=   0.8s
[CV] min_impurity_decrease=0.0001, min_samples_split=8 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=8, score=0.890, total=   0.8s
[CV] min_impurity_decrease=0.0001, min_samples_split=8 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=8, score=0.920, total=   0.8s
[CV] min_impurity_decrease=0.0001, min_samples_split=8 ...............
[CV]  min_impurity_decrease=0.0001, min_samples_split=8, score=0.904, 

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   31.8s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0001,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [19]:
print(grid.best_params_)
print(grid.best_score_)

{'min_impurity_decrease': 0.0001, 'min_samples_split': 2}
0.9057112068965517


In [20]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.2269056321772587, 'koi_model_snr'),
 (0.20520577006423435, 'koi_prad'),
 (0.1331415480980205, 'koi_teq'),
 (0.13182193575103482, 'koi_impact'),
 (0.11864712493042254, 'koi_depth'),
 (0.09289461431705683, 'koi_duration'),
 (0.09138337466197231, 'koi_insol')]

# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# model.save("voice_model_trained.h5")
import joblib
filename = 'random_forest.h5'
joblib.dump(rf, filename)

['random_forest.h5']